In [1]:
import copy
import os.path as osp

import torch

from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import mmcv
import numpy as np
import xml.etree.ElementTree as ET

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset


@DATASETS.register_module()
class HandBoneDataset(CustomDataset):

    CLASSES = ('radius', 'ulna', 'metacarpal1', 'metacarpal3', 'metacarpal5', 
               'nearly-phalanx1','nearly-phalanx3', 'nearly-phalanx5', 'middle-phalanx3', 'middle-phalanx5', 
              'far-phalanx1', 'far-phalanx3', 'far-phalanx5',)
    
    cat2lab = {val:i for i, val in enumerate(CLASSES)}
    lab2cat = {i:val for i, val in enumerate(CLASSES)}

    def parse_xml2dict(self, xml_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        infos = dict(filename = root.find('filename').text,
                     width = int(root.find('size')[0].text),
                     height = int(root.find('size')[1].text),
                    )
        labels = []
        boxs = []
        for member in root.findall('object'):
            label = self.cat2lab[member[0].text]
            box = (int(member[4][0].text),
                   int(member[4][1].text),
                   int(member[4][2].text),
                   int(member[4][3].text)
                  )
            labels.append(label)
            boxs.append(box)
        infos['ann'] = {'bboxes': np.array(boxs, dtype=np.float32), 'labels':np.array(labels, dtype=np.int64)}
        return infos
    
    def load_annotations(self, ann_file):
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []
        xml_prefix = self.img_prefix.replace('JPEGImages', 'Annotations')
        # convert annotations to middle format
        for image_id in image_list:
            # load annotations
            xml_file = osp.join(xml_prefix, image_id+'.xml')
            data_info = self.parse_xml2dict(xml_file)

            data_infos.append(data_info)

        return data_infos

In [6]:
from mmdet.apis import set_random_seed
from mmcv import Config
import os 

cfg = Config.fromfile('../configs/detectoRS_cascade_rcnn-hand-bone.py')

cfg.dataset_type = 'HandBoneDataset'
cfg.data_root = '/fdr/fdr/datasets/hand_bone'

cfg.data.train.type = 'HandBoneDataset'
cfg.data.train.data_root = '/fdr/fdr/datasets/hand_bone'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'JPEGImages'

cfg.data.test.type = 'HandBoneDataset'
cfg.data.test.data_root = '/fdr/fdr/datasets/hand_bone'
cfg.data.test.ann_file = 'val.txt'
cfg.data.test.img_prefix = 'JPEGImages'

cfg.data.val.type = 'HandBoneDataset'
cfg.data.val.data_root = '/fdr/fdr/datasets/hand_bone'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'JPEGImages'

cfg.work_dir = './hand_bone_exps'

cfg.optimizer.lr = 1e-1 
cfg.lr_config.warmup = 'linear'
cfg.log_config.interval = 1

cfg.evaluation.metric = 'mAP'
cfg.evaluation.interval = 2
cfg.checkpoint_config.interval = 5

cfg.seed = 79
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]

cfg.model.roi_head.bbox_head[0].num_classes = 13
cfg.model.roi_head.bbox_head[1].num_classes = 13
cfg.model.roi_head.bbox_head[2].num_classes = 13

cfg.model.backbone.frozen_stages = 0

In [8]:
cfg.total_epochs = 24
cfg.lr_config.step = [12, 18]

In [9]:
cfg.lr_config

{'policy': 'step',
 'warmup': 'linear',
 'warmup_iters': 500,
 'warmup_ratio': 0.001,
 'step': [12, 18]}

In [1]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

datasets = [build_dataset(cfg.data.train)]

In [2]:
model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)